In [11]:
# Update with minimum 3 schemas and at least 8 different tables across all SQLs

updated_data = [
    {
        "Request Number": 1,
        "Requirement": "Identify users from Samsung Members app who viewed at least 3 unique help articles and submitted a support ticket within the same week in the past 60 days.",
        "SQL Code": """\
SELECT 
  u.user_id,
  COUNT(DISTINCT ha.article_id) AS unique_articles_viewed,
  COUNT(st.ticket_id) AS support_tickets
FROM samsung_members.users u
JOIN samsung_support.help_article_views ha ON u.user_id = ha.user_id
JOIN samsung_support.support_tickets st ON u.user_id = st.user_id
WHERE ha.view_date BETWEEN CURRENT_DATE - INTERVAL '60 days' AND CURRENT_DATE
  AND st.created_at BETWEEN ha.view_date AND ha.view_date + INTERVAL '7 days'
GROUP BY u.user_id
HAVING COUNT(DISTINCT ha.article_id) >= 3 AND COUNT(st.ticket_id) >= 1;
"""
    },
    {
        "Request Number": 2,
        "Requirement": "For all users who have both Samsung Account and used the S Pen at least 5 times in the past month, list their device model, country, and avg session duration.",
        "SQL Code": """\
SELECT 
  sa.user_id,
  d.device_model,
  sa.country,
  AVG(s.session_duration_minutes) AS avg_session_duration
FROM samsung_accounts.account_users sa
JOIN samsung_device.s_pen_usage sp ON sa.user_id = sp.user_id
JOIN samsung_device.devices d ON sa.device_id = d.device_id
JOIN samsung_activity.app_sessions s ON sa.user_id = s.user_id
WHERE sp.usage_time >= CURRENT_DATE - INTERVAL '30 days'
GROUP BY sa.user_id, d.device_model, sa.country
HAVING COUNT(sp.usage_id) >= 5;
"""
    },
    {
        "Request Number": 3,
        "Requirement": "Get the top 3 most accessed Samsung cloud services per region in Q1 2025, along with access counts.",
        "SQL Code": """\
SELECT 
  region,
  service_name,
  access_count
FROM (
  SELECT 
    u.region,
    cs.service_name,
    COUNT(*) AS access_count,
    RANK() OVER (PARTITION BY u.region ORDER BY COUNT(*) DESC) AS rank
  FROM samsung_cloud.cloud_service_usage cs
  JOIN samsung_accounts.account_users u ON cs.user_id = u.user_id
  WHERE cs.access_time BETWEEN '2025-01-01' AND '2025-03-31'
  GROUP BY u.region, cs.service_name
) ranked_services
WHERE rank <= 3;
"""
    },
    {
        "Request Number": 4,
        "Requirement": "Track login-to-purchase funnel for Samsung Store App users who logged in and made a purchase within 48 hours in the past 45 days, also fetch product category and payment mode.",
        "SQL Code": """\
SELECT 
  l.user_id,
  MIN(l.login_time) AS first_login,
  MIN(p.purchase_time) AS first_purchase,
  pr.product_category,
  pay.payment_method
FROM samsung_store.logins l
JOIN samsung_store.purchases p 
  ON l.user_id = p.user_id AND p.purchase_time BETWEEN l.login_time AND l.login_time + INTERVAL '48 hours'
JOIN samsung_store.products pr ON p.product_id = pr.product_id
JOIN samsung_payments.transactions pay ON p.payment_id = pay.payment_id
WHERE l.login_time >= CURRENT_DATE - INTERVAL '45 days'
GROUP BY l.user_id, pr.product_category, pay.payment_method;
"""
    }
]



In [12]:
df.head()

,Request Number,Requirement,SQL Code
0,1,Identify users from Samsung Members app who vi...,"SELECT \n u.user_id,\n COUNT(DISTINCT ha.art..."
1,2,For all users who have both Samsung Account an...,"SELECT \n sa.user_id,\n d.device_model,\n s..."
2,3,Get the top 3 most accessed Samsung cloud serv...,"SELECT \n region,\n service_name,\n access_..."
3,4,Track login-to-purchase funnel for Samsung Sto...,"SELECT \n l.user_id,\n MIN(l.login_time) AS ..."


In [13]:

df.to_csv("samsung_requests_complex.csv")




In [14]:
df

,Request Number,Requirement,SQL Code
0,1,Identify users from Samsung Members app who vi...,"SELECT \n u.user_id,\n COUNT(DISTINCT ha.art..."
1,2,For all users who have both Samsung Account an...,"SELECT \n sa.user_id,\n d.device_model,\n s..."
2,3,Get the top 3 most accessed Samsung cloud serv...,"SELECT \n region,\n service_name,\n access_..."
3,4,Track login-to-purchase funnel for Samsung Sto...,"SELECT \n l.user_id,\n MIN(l.login_time) AS ..."


In [15]:
# # Merge metadata into the original SQL + requirements DataFrame
# df_combined = df_updated.copy()
df["Metadata"] = [
    """Schemas Used: samsung_members, samsung_support | Tables Used: users, help_article_views, support_tickets | Joins: users.user_id = help_article_views.user_id, users.user_id = support_tickets.user_id | Filters: view_date within 60 days, ticket created within 7 days of article view | Metrics: Count of articles, Count of tickets | Grouping: user_id | Conditions: >=3 articles and >=1 ticket""",
    """Schemas Used: samsung_accounts, samsung_device, samsung_activity | Tables Used: account_users, s_pen_usage, devices, app_sessions | Joins: account_users.user_id = s_pen_usage.user_id, account_users.device_id = devices.device_id, account_users.user_id = app_sessions.user_id | Filters: S Pen usage in last 30 days | Metrics: Avg session duration, Count of S Pen usage | Grouping: user_id, device_model, country | Conditions: >=5 S Pen usages""",
    """Schemas Used: samsung_cloud, samsung_accounts | Tables Used: cloud_service_usage, account_users | Joins: cloud_service_usage.user_id = account_users.user_id | Filters: Q1 2025 | Metrics: Count of accesses | Grouping: region, service_name | Conditions: Top 3 services by region (RANK)""",
    """Schemas Used: samsung_store, samsung_payments | Tables Used: logins, purchases, products, transactions | Joins: logins.user_id = purchases.user_id, purchases.product_id = products.product_id, purchases.payment_id = transactions.payment_id | Filters: logins in last 45 days, purchase within 48h of login | Metrics: First login, first purchase, product category, payment method | Grouping: user_id, product_category, payment_method | Conditions: login-to-purchase within 48 hours"""
]

# # Save to new CSV
# final_csv_with_metadata = "/mnt/data/samsung_requests_with_full_metadata.csv"
# df_combined.to_csv(final_csv_with_metadata, index=False)

# final_csv_with_metadata


In [16]:
df

,Request Number,Requirement,SQL Code,Metadata
0,1,Identify users from Samsung Members app who vi...,"SELECT \n u.user_id,\n COUNT(DISTINCT ha.art...","Schemas Used: samsung_members, samsung_support..."
1,2,For all users who have both Samsung Account an...,"SELECT \n sa.user_id,\n d.device_model,\n s...","Schemas Used: samsung_accounts, samsung_device..."
2,3,Get the top 3 most accessed Samsung cloud serv...,"SELECT \n region,\n service_name,\n access_...","Schemas Used: samsung_cloud, samsung_accounts ..."
3,4,Track login-to-purchase funnel for Samsung Sto...,"SELECT \n l.user_id,\n MIN(l.login_time) AS ...","Schemas Used: samsung_store, samsung_payments ..."


In [18]:
import re

In [20]:
# Step 3: Helper functions to extract structured metadata
def extract_schemas_fixed(sql):
    matches = re.findall(r'FROM (\w+)\.|JOIN (\w+)\.', sql)
    flat = [m for tup in matches for m in tup if m]
    return sorted(set(flat))

def extract_tables(sql):
    return sorted(set([a or b for a, b in re.findall(r'FROM (\w+\.\w+)|JOIN (\w+\.\w+)', sql)]))

def extract_columns(sql):
    matches = re.findall(r'\b(\w+\.\w+)', sql)
    return sorted(set(matches))

def classify_tag(requirement):
    if 'support' in requirement.lower():
        return 'support_activity'
    elif 'funnel' in requirement.lower():
        return 'funnel_analysis'
    elif 's pen' in requirement.lower() or 'session' in requirement.lower():
        return 'feature_usage'
    elif 'cloud' in requirement.lower():
        return 'cloud_service_ranking'
    else:
        return 'general_analysis'

# Step 4: Apply enhancements
df["Schemas Used"] = df["SQL Code"].apply(lambda sql: ', '.join(extract_schemas_fixed(sql)))
df["Tables Used"] = df["SQL Code"].apply(lambda sql: ', '.join(extract_tables(sql)))
df["Columns Used"] = df["SQL Code"].apply(lambda sql: ', '.join(extract_columns(sql)))
df["Tags"] = df["Requirement"].apply(classify_tag)

In [21]:
df

,Request Number,Requirement,SQL Code,Metadata,Schemas Used,Tables Used,Columns Used,Tags
0,1,Identify users from Samsung Members app who vi...,"SELECT \n u.user_id,\n COUNT(DISTINCT ha.art...","Schemas Used: samsung_members, samsung_support...",,,"ha.article_id, ha.user_id, ha.view_date, st.cr...",support_activity
1,2,For all users who have both Samsung Account an...,"SELECT \n sa.user_id,\n d.device_model,\n s...","Schemas Used: samsung_accounts, samsung_device...",,,"d.device_id, d.device_model, s.session_duratio...",feature_usage
2,3,Get the top 3 most accessed Samsung cloud serv...,"SELECT \n region,\n service_name,\n access_...","Schemas Used: samsung_cloud, samsung_accounts ...",,,"cs.access_time, cs.service_name, cs.user_id, u...",cloud_service_ranking
3,4,Track login-to-purchase funnel for Samsung Sto...,"SELECT \n l.user_id,\n MIN(l.login_time) AS ...","Schemas Used: samsung_store, samsung_payments ...",,,"l.login_time, l.user_id, p.purchase_time, p.us...",funnel_analysis


In [22]:
# Updated schema and table extractor
def extract_schemas_and_tables(sql):
    matches = re.findall(r'\b(\w+)\.(\w+)', sql)
    schemas = sorted(set([m[0] for m in matches]))
    tables = sorted(set(['.'.join(m) for m in matches]))
    return schemas, tables

# Simple keyword-based tag generator (can be replaced with embedding similarity or classification)
def generate_tag(text):
    keywords = {
        'support': 'support',
        'funnel': 'funnel',
        'cloud': 'cloud',
        's pen': 'device_usage',
        'session': 'user_behavior',
        'login': 'authentication',
        'purchase': 'commerce',
        'product': 'product',
        'ticket': 'support',
        'help': 'support'
    }
    text_lower = text.lower()
    for k, v in keywords.items():
        if k in text_lower:
            return v
    return 'general'

# Apply correct schema/table extraction and tag generation
schemas_list = []
tables_list = []

for sql in df["SQL Code"]:
    schemas, tables = extract_schemas_and_tables(sql)
    schemas_list.append(', '.join(schemas))
    tables_list.append(', '.join(tables))

df["Schemas Used"] = schemas_list
df["Tables Used"] = tables_list
df["Columns Used"] = df["SQL Code"].apply(lambda sql: ', '.join(extract_columns(sql)))
df["Tags"] = df["Requirement"].apply(generate_tag)

In [23]:
df

,Request Number,Requirement,SQL Code,Metadata,Schemas Used,Tables Used,Columns Used,Tags
0,1,Identify users from Samsung Members app who vi...,"SELECT \n u.user_id,\n COUNT(DISTINCT ha.art...","Schemas Used: samsung_members, samsung_support...","ha, st, u","ha.article_id, ha.user_id, ha.view_date, st.cr...","ha.article_id, ha.user_id, ha.view_date, st.cr...",support
1,2,For all users who have both Samsung Account an...,"SELECT \n sa.user_id,\n d.device_model,\n s...","Schemas Used: samsung_accounts, samsung_device...","d, s, sa, sp","d.device_id, d.device_model, s.session_duratio...","d.device_id, d.device_model, s.session_duratio...",device_usage
2,3,Get the top 3 most accessed Samsung cloud serv...,"SELECT \n region,\n service_name,\n access_...","Schemas Used: samsung_cloud, samsung_accounts ...","cs, u","cs.access_time, cs.service_name, cs.user_id, u...","cs.access_time, cs.service_name, cs.user_id, u...",cloud
3,4,Track login-to-purchase funnel for Samsung Sto...,"SELECT \n l.user_id,\n MIN(l.login_time) AS ...","Schemas Used: samsung_store, samsung_payments ...","l, p","l.login_time, l.user_id, p.purchase_time, p.us...","l.login_time, l.user_id, p.purchase_time, p.us...",funnel


In [25]:
def infer_relationships(sql):
    # Extract JOIN conditions
    joins = re.findall(r'JOIN\s+(\w+\.\w+)\s+\w+\s+ON\s+([\w\.]+)\s*=\s*([\w\.]+)', sql)
    relationships = []
    for table, col1, col2 in joins:
        relationships.append(f"{col1} = {col2}")
    return '; '.join(relationships) if relationships else "No explicit joins"

# Apply to DataFrame
df["Relationships"] = df["SQL Code"].apply(infer_relationships)

In [26]:
df

,Request Number,Requirement,SQL Code,Metadata,Schemas Used,Tables Used,Columns Used,Tags,Relationships
0,1,Identify users from Samsung Members app who vi...,"SELECT \n u.user_id,\n COUNT(DISTINCT ha.art...","Schemas Used: samsung_members, samsung_support...","ha, st, u","ha.article_id, ha.user_id, ha.view_date, st.cr...","ha.article_id, ha.user_id, ha.view_date, st.cr...",support,No explicit joins
1,2,For all users who have both Samsung Account an...,"SELECT \n sa.user_id,\n d.device_model,\n s...","Schemas Used: samsung_accounts, samsung_device...","d, s, sa, sp","d.device_id, d.device_model, s.session_duratio...","d.device_id, d.device_model, s.session_duratio...",device_usage,No explicit joins
2,3,Get the top 3 most accessed Samsung cloud serv...,"SELECT \n region,\n service_name,\n access_...","Schemas Used: samsung_cloud, samsung_accounts ...","cs, u","cs.access_time, cs.service_name, cs.user_id, u...","cs.access_time, cs.service_name, cs.user_id, u...",cloud,No explicit joins
3,4,Track login-to-purchase funnel for Samsung Sto...,"SELECT \n l.user_id,\n MIN(l.login_time) AS ...","Schemas Used: samsung_store, samsung_payments ...","l, p","l.login_time, l.user_id, p.purchase_time, p.us...","l.login_time, l.user_id, p.purchase_time, p.us...",funnel,No explicit joins


In [28]:

def extract_schema_table_column(sql):
    # Get all table and column combinations
    matches = re.findall(r'\b(\w+)\.(\w+)', sql)
    return matches

def build_structured_relationship(sql):
    table_columns = extract_schema_table_column(sql)
    relationships = {}

    # Group columns by schema.table
    for schema, table_or_col in table_columns:
        key = schema + '.' + table_or_col
        if '.' in key:
            relationships.setdefault(schema, {}).setdefault(table_or_col, set())

    # Add columns used for each schema.table
    for full_col in re.findall(r'\b(\w+\.\w+)\.(\w+)', sql):
        schema_table = f"{full_col[0]}.{full_col[1]}"
        colname = full_col[2] if len(full_col) > 2 else None
        schema, table = full_col[0], full_col[1]
        if schema in relationships and table in relationships[schema]:
            relationships[schema][table].add(colname)

    # Join structure
    join_matches = re.findall(r'JOIN\s+(\w+\.\w+)\s+\w+\s+ON\s+([\w\.]+)\s*=\s*([\w\.]+)', sql)
    joins = [f"{j[1]} = {j[2]}" for j in join_matches]

    # Build structured string
    structure = []
    for schema, tables in relationships.items():
        for table, columns in tables.items():
            structure.append(f"{schema}.{table} -> Columns: [ ]")  # Skipping columns for now due to limitations

    if joins:
        structure.append("Joins: " + "; ".join(joins))

    return " | ".join(structure)


def structured_relationship_fallback(sql):
    schemas_and_tables = extract_schemas_and_tables(sql)
    columns = extract_columns(sql)
    join_matches = re.findall(r'JOIN\s+(\w+\.\w+)\s+\w+\s+ON\s+([\w\.]+)\s*=\s*([\w\.]+)', sql)
    join_texts = [f"{j[1]} = {j[2]}" for j in join_matches]

    structure = []
    for tbl in schemas_and_tables[1]:  # Tables
        table_columns = [col for col in columns if col.startswith(tbl)]
        structure.append(f"{tbl} -> Columns: [{', '.join([col.split('.')[-1] for col in table_columns])}]")

    if join_texts:
        structure.append("Joins: " + "; ".join(join_texts))

    return " | ".join(structure)

# Apply to DataFrame
df["Structured Relationships"] = df["SQL Code"].apply(structured_relationship_fallback)



In [29]:
df

,Request Number,Requirement,SQL Code,Metadata,Schemas Used,Tables Used,Columns Used,Tags,Relationships,Structured Relationships
0,1,Identify users from Samsung Members app who vi...,"SELECT \n u.user_id,\n COUNT(DISTINCT ha.art...","Schemas Used: samsung_members, samsung_support...","ha, st, u","ha.article_id, ha.user_id, ha.view_date, st.cr...","ha.article_id, ha.user_id, ha.view_date, st.cr...",support,No explicit joins,ha.article_id -> Columns: [article_id] | ha.us...
1,2,For all users who have both Samsung Account an...,"SELECT \n sa.user_id,\n d.device_model,\n s...","Schemas Used: samsung_accounts, samsung_device...","d, s, sa, sp","d.device_id, d.device_model, s.session_duratio...","d.device_id, d.device_model, s.session_duratio...",device_usage,No explicit joins,d.device_id -> Columns: [device_id] | d.device...
2,3,Get the top 3 most accessed Samsung cloud serv...,"SELECT \n region,\n service_name,\n access_...","Schemas Used: samsung_cloud, samsung_accounts ...","cs, u","cs.access_time, cs.service_name, cs.user_id, u...","cs.access_time, cs.service_name, cs.user_id, u...",cloud,No explicit joins,cs.access_time -> Columns: [access_time] | cs....
3,4,Track login-to-purchase funnel for Samsung Sto...,"SELECT \n l.user_id,\n MIN(l.login_time) AS ...","Schemas Used: samsung_store, samsung_payments ...","l, p","l.login_time, l.user_id, p.purchase_time, p.us...","l.login_time, l.user_id, p.purchase_time, p.us...",funnel,No explicit joins,l.login_time -> Columns: [login_time] | l.user...


In [31]:
!pip install -q sqlglot


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: C:\Users\SaikishorePichili\anaconda3\python.exe -m pip install --upgrade pip


In [35]:
sqlglot_data = df["SQL Code"].apply(extract_sqlglot_metadata)

In [43]:
def extract_sqlglot_metadata_debug(sql_text, row_id=None):
    try:
        cleaned_sql = ' '.join(sql_text.strip().split())
        parsed = parse_one(cleaned_sql, read='ansi')

        real_tables = [t.sql() for t in parsed.find_all(Table) if not t.find_ancestor(CTE)]
        ctes = list(parsed.ctes.keys())
        columns = [c.sql() for c in parsed.find_all(Column)]

        joins = []
        for join in parsed.find_all(Join):
            if join.args.get("on"):
                joins.append(join.args["on"].sql(dialect='ansi'))

        return {
            "Extracted Tables": ', '.join(sorted(set(real_tables))),
            "CTEs": ', '.join(ctes),
            "Used Columns": ', '.join(sorted(set(columns))),
            "Join Conditions": '; '.join(joins) if joins else "None"
        }

    except Exception as e:
        print(f"❌ Error parsing row {row_id}:\n{sql_text}\n---\n{e}\n")
        return {
            "Extracted Tables": f"Error: {e}",
            "CTEs": "Error",
            "Used Columns": "Error",
            "Join Conditions": "Error"
        }


In [44]:
sqlglot_data = df["SQL Code"].apply(lambda x: extract_sqlglot_metadata_debug(x, row_id=df[df["SQL Code"] == x].index[0]))
sqlglot_data = sqlglot_data.apply(pd.Series)
df = pd.concat([df, sqlglot_data], axis=1)


❌ Error parsing row 0:
SELECT 
  u.user_id,
  COUNT(DISTINCT ha.article_id) AS unique_articles_viewed,
  COUNT(st.ticket_id) AS support_tickets
FROM samsung_members_users u
JOIN samsung_help_article_views ha ON u.user_id = ha.user_id
JOIN samsung_support_tickets st ON u.user_id = st.user_id
WHERE ha.view_date BETWEEN CURRENT_DATE - INTERVAL '60 days' AND CURRENT_DATE
  AND st.created_at BETWEEN ha.view_date AND ha.view_date + INTERVAL '7 days'
GROUP BY u.user_id
HAVING COUNT(DISTINCT ha.article_id) >= 3 AND COUNT(st.ticket_id) >= 1;

---
Unknown dialect 'ansi'.

❌ Error parsing row 1:
SELECT 
  sa.user_id,
  d.device_model,
  sa.country,
  AVG(s.session_duration_minutes) AS avg_session_duration
FROM samsung_account_users sa
JOIN s_pen_usage sp ON sa.user_id = sp.user_id
JOIN samsung_devices d ON sa.device_id = d.device_id
JOIN samsung_app_sessions s ON sa.user_id = s.user_id
WHERE sp.usage_time >= CURRENT_DATE - INTERVAL '30 days'
GROUP BY sa.user_id, d.device_model, sa.country
HAVING 

In [40]:
for i, sql in enumerate(df["SQL Code"]):
    try:
        cleaned_sql = ' '.join(sql.strip().split())
        parse_one(cleaned_sql)
    except Exception as e:
        print(f"Row {i} failed to parse:\n{cleaned_sql}\nError: {e}")


In [46]:
from sqlglot import parse_one

sql = """
SELECT u.user_id, COUNT(DISTINCT ha.article_id) AS unique_articles_viewed, COUNT(st.ticket_id) AS support_tickets
FROM samsung_members.users u
JOIN samsung_support.help_article_views ha ON u.user_id = ha.user_id
JOIN samsung_support.support_tickets st ON u.user_id = st.user_id
WHERE ha.view_date BETWEEN CURRENT_DATE - INTERVAL '60 days' AND CURRENT_DATE
AND st.created_at BETWEEN ha.view_date AND ha.view_date + INTERVAL '7 days'
GROUP BY u.user_id
HAVING COUNT(DISTINCT ha.article_id) >= 3 AND COUNT(st.ticket_id) >= 1;
"""

# Strip and normalize
cleaned = ' '.join(sql.strip().split())

# Try parsing
try:
    parsed = parse_one(cleaned)
    print(parsed)
except Exception as e:
    print("Failed to parse:", e)


SELECT u.user_id, COUNT(DISTINCT ha.article_id) AS unique_articles_viewed, COUNT(st.ticket_id) AS support_tickets FROM samsung_members.users AS u JOIN samsung_support.help_article_views AS ha ON u.user_id = ha.user_id JOIN samsung_support.support_tickets AS st ON u.user_id = st.user_id WHERE ha.view_date BETWEEN CURRENT_DATE - INTERVAL '60' DAYS AND CURRENT_DATE AND st.created_at BETWEEN ha.view_date AND ha.view_date + INTERVAL '7' DAYS GROUP BY u.user_id HAVING COUNT(DISTINCT ha.article_id) >= 3 AND COUNT(st.ticket_id) >= 1


In [48]:
from sqlglot.expressions import Table, Column, Join, CTE

# Parsed object
parsed = parse_one(cleaned)

# Extract metadata
real_tables = [t.sql() for t in parsed.find_all(Table) if not t.find_ancestor(CTE)]
ctes = parsed.ctes or []
cte_names = [cte.alias_or_name for cte in ctes]

columns = [c.sql() for c in parsed.find_all(Column)]

joins = []
for join in parsed.find_all(Join):
    if join.args.get("on"):
        joins.append(join.args["on"].sql())

# Display results
print("🗂️ Extracted Tables:", real_tables)
print("📄 CTEs:", cte_names)
print("🔢 Used Columns:", columns)
print("🔗 Join Conditions:", joins)


🗂️ Extracted Tables: ['samsung_members.users AS u', 'samsung_support.help_article_views AS ha', 'samsung_support.support_tickets AS st']
📄 CTEs: []
🔢 Used Columns: ['u.user_id', 'u.user_id', 'st.ticket_id', 'u.user_id', 'ha.user_id', 'u.user_id', 'st.user_id', 'ha.article_id', 'ha.view_date', 'st.created_at', 'ha.view_date', 'ha.view_date', 'st.ticket_id', 'ha.article_id']
🔗 Join Conditions: ['u.user_id = ha.user_id', 'u.user_id = st.user_id']


In [49]:
def extract_sqlglot_metadata(sql_text):
    try:
        cleaned = ' '.join(sql_text.strip().split())
        parsed = parse_one(cleaned)

        real_tables = [t.sql() for t in parsed.find_all(Table) if not t.find_ancestor(CTE)]
        ctes = list(parsed.ctes.keys())
        columns = [c.sql() for c in parsed.find_all(Column)]
        joins = []
        for join in parsed.find_all(Join):
            if join.args.get("on"):
                joins.append(join.args["on"].sql())

        return {
            "Extracted Tables": ', '.join(sorted(set(real_tables))),
            "CTEs": ', '.join(ctes),
            "Used Columns": ', '.join(sorted(set(columns))),
            "Join Conditions": '; '.join(joins) if joins else "None"
        }

    except Exception as e:
        return {
            "Extracted Tables": f"Error: {e}",
            "CTEs": "Error",
            "Used Columns": "Error",
            "Join Conditions": "Error"
        }

# Apply to DataFrame
sqlglot_data = df["SQL Code"].apply(extract_sqlglot_metadata).apply(pd.Series)
df = pd.concat([df, sqlglot_data], axis=1)


In [50]:
df

,Request Number,Requirement,SQL Code,Metadata,Schemas Used,Tables Used,Columns Used,Tags,Relationships,Structured Relationships,...,Used Columns,Join Conditions,Extracted Tables,CTEs,Used Columns,Join Conditions,Extracted Tables,CTEs,Used Columns,Join Conditions
0,1,Identify users from Samsung Members app who vi...,"SELECT \n u.user_id,\n COUNT(DISTINCT ha.art...","Schemas Used: samsung_members, samsung_support...","ha, st, u","ha.article_id, ha.user_id, ha.view_date, st.cr...","ha.article_id, ha.user_id, ha.view_date, st.cr...",support,No explicit joins,ha.article_id -> Columns: [article_id] | ha.us...,...,Error,Error,Error: Unknown dialect 'ansi'.,Error,Error,Error,Error: 'list' object has no attribute 'keys',Error,Error,Error
1,2,For all users who have both Samsung Account an...,"SELECT \n sa.user_id,\n d.device_model,\n s...","Schemas Used: samsung_accounts, samsung_device...","d, s, sa, sp","d.device_id, d.device_model, s.session_duratio...","d.device_id, d.device_model, s.session_duratio...",device_usage,No explicit joins,d.device_id -> Columns: [device_id] | d.device...,...,Error,Error,Error: Unknown dialect 'ansi'.,Error,Error,Error,Error: 'list' object has no attribute 'keys',Error,Error,Error
2,3,Get the top 3 most accessed Samsung cloud serv...,"SELECT \n region,\n service_name,\n access_...","Schemas Used: samsung_cloud, samsung_accounts ...","cs, u","cs.access_time, cs.service_name, cs.user_id, u...","cs.access_time, cs.service_name, cs.user_id, u...",cloud,No explicit joins,cs.access_time -> Columns: [access_time] | cs....,...,Error,Error,Error: Unknown dialect 'ansi'.,Error,Error,Error,Error: 'list' object has no attribute 'keys',Error,Error,Error
3,4,Track login-to-purchase funnel for Samsung Sto...,"SELECT \n l.user_id,\n MIN(l.login_time) AS ...","Schemas Used: samsung_store, samsung_payments ...","l, p","l.login_time, l.user_id, p.purchase_time, p.us...","l.login_time, l.user_id, p.purchase_time, p.us...",funnel,No explicit joins,l.login_time -> Columns: [login_time] | l.user...,...,Error,Error,Error: Unknown dialect 'ansi'.,Error,Error,Error,Error: 'list' object has no attribute 'keys',Error,Error,Error


In [52]:
error_columns = [
    col for col in df.columns
    if df[col].apply(lambda x: isinstance(x, str) and x.startswith("Error")).any()
]

# Drop those columns
df_cleaned = df.drop(columns=error_columns)

In [63]:
df

,Request Number,Requirement,SQL Code,Metadata,Schemas Used,Tables Used,Columns Used,Tags,Structured Relationships
0,1,Identify users from Samsung Members app who vi...,"SELECT \n u.user_id,\n COUNT(DISTINCT ha.art...","Schemas Used: samsung_members, samsung_support...","ha, st, u","ha.article_id, ha.user_id, ha.view_date, st.cr...","ha.article_id, ha.user_id, ha.view_date, st.cr...",support,ha.article_id -> Columns: [article_id] | ha.us...
1,2,For all users who have both Samsung Account an...,"SELECT \n sa.user_id,\n d.device_model,\n s...","Schemas Used: samsung_accounts, samsung_device...","d, s, sa, sp","d.device_id, d.device_model, s.session_duratio...","d.device_id, d.device_model, s.session_duratio...",device_usage,d.device_id -> Columns: [device_id] | d.device...
2,3,Get the top 3 most accessed Samsung cloud serv...,"SELECT \n region,\n service_name,\n access_...","Schemas Used: samsung_cloud, samsung_accounts ...","cs, u","cs.access_time, cs.service_name, cs.user_id, u...","cs.access_time, cs.service_name, cs.user_id, u...",cloud,cs.access_time -> Columns: [access_time] | cs....
3,4,Track login-to-purchase funnel for Samsung Sto...,"SELECT \n l.user_id,\n MIN(l.login_time) AS ...","Schemas Used: samsung_store, samsung_payments ...","l, p","l.login_time, l.user_id, p.purchase_time, p.us...","l.login_time, l.user_id, p.purchase_time, p.us...",funnel,l.login_time -> Columns: [login_time] | l.user...


In [62]:
df.drop(columns=['Relationships'], inplace=True)

In [64]:
import networkx as nx

# Initialize a directed graph
G = nx.DiGraph()

In [65]:
for _, row in df_cleaned.iterrows():
    if pd.notnull(row.get("Tables Used")) and pd.notnull(row.get("Columns Used")):
        tables = [tbl.strip() for tbl in row["Tables Used"].split(",")]
        columns = [col.strip() for col in row["Columns Used"].split(",")]
        
        for tbl in tables:
            if "." in tbl:
                schema, table = tbl.split(".", 1)
                G.add_edge(schema, tbl, label="has_table")
            else:
                schema = "unknown_schema"
                G.add_edge(schema, tbl, label="has_table")
        
        for col in columns:
            if "." in col:
                tbl, column = col.split(".", 1)
                # Attempt to find fully qualified table from previously added tables
                matching_tables = [t for t in tables if t.endswith(tbl)]
                if matching_tables:
                    full_tbl = matching_tables[0]
                    G.add_edge(full_tbl, col, label="has_column")

# Convert relationships into a DataFrame for export
edges_data = [(u, v, d["label"]) for u, v, d in G.edges(data=True)]
relationships_df = pd.DataFrame(edges_data, columns=["From", "To", "Relationship"])

In [66]:
relationships_df

,From,To,Relationship
0,ha,ha.article_id,has_table
1,ha,ha.user_id,has_table
2,ha,ha.view_date,has_table
3,st,st.created_at,has_table
4,st,st.ticket_id,has_table
5,st,st.user_id,has_table
6,u,u.user_id,has_table
7,u,u.region,has_table
8,d,d.device_id,has_table
9,d,d.device_model,has_table


In [ ]:
import pandas as pd
from sqlglot import parse_one
from sqlglot.expressions import Table, Column, Join, CTE

# Load your CSV
df = pd.read_csv("your_input.csv")

def universal_relationship_extractor(sql):
    try:
        cleaned_sql = ' '.join(sql.strip().split())
        parsed = parse_one(cleaned_sql)

        tables = [t.sql() for t in parsed.find_all(Table) if not t.find_ancestor(CTE)]
        columns = [c.sql() for c in parsed.find_all(Column)]
        joins = [j.args["on"].sql() for j in parsed.find_all(Join) if j.args.get("on")]

        relationships = []
        for tbl in tables:
            if '.' in tbl:
                schema, table = tbl.split('.', 1)
                relationships.append(f"{schema} -> {tbl}")
        for col in columns:
            if '.' in col:
                tbl_alias, column = col.split('.', 1)
                match = next((t for t in tables if t.endswith(tbl_alias)), tbl_alias)
                relationships.append(f"{match} -> {col}")
        if joins:
            relationships.append("Joins: " + "; ".join(joins))

        return " | ".join(relationships)

    except Exception as e:
        return f"Error: {str(e)}"

# Apply it
df["Universal Relationships"] = df["SQL Code"].apply(universal_relationship_extractor)

# Save result
df.to_csv("samsung_requests_universal_relationships.csv", index=False)
